In [25]:
import sys
sys.path.append("/home/brunoperdigao/repos/Plan-Classification-Topologic-DGL")

In [26]:
# Import Libraries
from topologicpy.DGL import DGL
from topologicpy.Dictionary import Dictionary
from topologicpy.Topology import Topology
from topologicpy.Vertex import Vertex
from topologicpy.Graph import Graph
from topologicpy.Plotly import Plotly
import dgl
import torch
import random
from dgl import save_graphs, load_graphs

In [27]:
import dgl

In [28]:
pwd

'/home/bruno_perdigao/repos/Plan-Classification-Topologic-DGL'

In [29]:
path = "./data"
dataset = DGL.DatasetByCSVPath(path, numberOfGraphClasses=0,
                           nodeATTRKey='feat', edgeATTRKey='feat',
                           nodeOneHotEncode=False, nodeFeaturesCategories=[],
                           edgeOneHotEncode=False, edgeFeaturesCategories=[], addSelfLoop=False)
print(dataset)

Done saving data into cached files.
Dataset("GraphDGL", num_graphs=100, save_path=/home/bruno_perdigao/.dgl/GraphDGL)


In [30]:
train_data, val_data, test_data = dgl.data.utils.split_dataset(dataset, [0.7, 0.1, 0.2])

In [31]:
train_data.dataset

Dataset("GraphDGL", num_graphs=100, save_path=/home/bruno_perdigao/.dgl/GraphDGL)

---

In [32]:
# Contruct a two-layer GNN model
import dgl.nn as dglnn
import torch.nn as nn
import torch.nn.functional as F
class SAGE(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats):
        super().__init__()
        self.conv1 = dglnn.SAGEConv(
            in_feats=in_feats, out_feats=hid_feats, aggregator_type='mean')
        self.conv2 = dglnn.SAGEConv(
            in_feats=hid_feats, out_feats=out_feats, aggregator_type='mean')

    def forward(self, graph, inputs):
        # inputs are features of nodes
        h = self.conv1(graph, inputs)
        h = F.relu(h)
        h = self.conv2(graph, h)
        return h

In [33]:
def evaluate(model, graph, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(graph, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [34]:
# # print(len(dataset.graphs))
# for i, graph in enumerate(dataset.graphs):
#     # print(i)
#     node_features = graph.ndata['feat']
#     # print(node_features)
#     node_labels = graph.ndata['label']
#     # print(node_labels)
#     train_mask = graph.ndata['train_mask']
#     valid_mask = graph.ndata['val_mask']
#     test_mask = graph.ndata['test_mask']
#     n_features = node_features.shape[1]
#     # print(n_features)
#     n_labels = int(node_labels.max().item() + 1)
#     # print(n_features)

In [40]:
n_features = 5
model = SAGE(in_feats=n_features, hid_feats=100, out_feats=12)
opt = torch.optim.Adam(model.parameters())

for epoch in range(50):
    for i, graph in enumerate(dataset.graphs):
        node_features = graph.ndata['feat']
        node_labels = graph.ndata['label']
        train_mask = graph.ndata['train_mask']
        valid_mask = graph.ndata['val_mask']
        test_mask = graph.ndata['test_mask']
        n_features = node_features.shape[1]
        n_labels = int(node_labels.max().item() + 1)
        
        model.train()
        # forward propagation by using all nodes
        logits = model(graph, node_features)
        # compute loss
        loss = F.cross_entropy(logits[train_mask], node_labels[train_mask])
        # compute validation accuracy
        acc = evaluate(model, graph, node_features, node_labels, valid_mask)
        # backward propagation
        opt.zero_grad()
        loss.backward()
        opt.step()
        print(loss.item())
    
        # Save model if necessary.  Omitted in this example.

55.525726318359375
53.158409118652344
34.64437484741211
35.32785415649414
34.71891784667969
32.623207092285156
37.766357421875
29.17270851135254
31.56928825378418
22.717851638793945
38.4640998840332
18.631593704223633
25.508037567138672
19.983797073364258
17.526782989501953
17.792699813842773
18.558887481689453
26.313182830810547
15.19589900970459
19.915809631347656
19.839113235473633
13.823360443115234
14.552793502807617
12.948029518127441
9.470531463623047
12.35519790649414
7.258391857147217
11.126622200012207
10.707148551940918
9.906314849853516
16.668472290039062
13.874849319458008
10.306187629699707
7.889223575592041
10.331501007080078
10.449369430541992
8.337197303771973
7.585925579071045
6.337114334106445
6.7475786209106445
6.212055206298828
4.925685882568359
4.2007622718811035
4.061246871948242
4.23414945602417
5.118101119995117
3.433654308319092
4.043692111968994
3.181990385055542
4.373391151428223
20.172115325927734
19.85472869873047
13.459451675415039
18.334379196166992
19.9

In [41]:
DGL.ModelClassifyNodes(model, test_data.dataset)

{'alllabels': [[8, 1, 9, 2, 4, 6, 0, 3, 11, 7],
  [2, 7, 0, 1, 11, 4, 7, 3, 6, 8, 6, 9],
  [4, 8, 7, 2, 11, 9, 3, 11, 6, 0, 9, 1, 7, 7],
  [0, 11, 9, 7, 7, 8, 2, 4, 1, 3, 11, 6, 9, 7],
  [3, 9, 11, 7, 6, 9, 0, 1, 7, 2, 11, 4, 7, 8],
  [3, 11, 7, 8, 0, 11, 6, 7, 4, 7, 7, 9, 1, 2, 9, 6],
  [8, 6, 2, 3, 4, 1, 9, 7, 11, 0],
  [9, 3, 2, 7, 6, 4, 7, 11, 6, 8, 0, 1],
  [3, 9, 6, 7, 2, 0, 4, 7, 1, 6, 11, 8],
  [2, 9, 7, 11, 0, 6, 1, 7, 11, 4, 9, 7, 8, 3],
  [3, 7, 6, 1, 11, 2, 8, 4, 9, 0],
  [6, 11, 9, 3, 8, 2, 1, 11, 7, 0, 4, 7, 7, 7, 9, 6],
  [3, 9, 0, 4, 11, 8, 6, 2, 7, 1],
  [2, 3, 7, 6, 8, 9, 0, 1, 11, 4],
  [7, 1, 3, 9, 7, 0, 11, 11, 6, 9, 2, 8, 7, 4],
  [7, 9, 1, 0, 11, 8, 7, 9, 6, 4, 6, 2, 11, 3, 7, 7],
  [9, 7, 7, 6, 2, 11, 1, 3, 4, 0, 8, 11, 9, 7],
  [2, 3, 8, 9, 11, 6, 6, 7, 1, 4, 0, 7],
  [2, 11, 9, 1, 6, 0, 4, 7, 8, 3, 7],
  [4, 1, 11, 2, 6, 7, 8, 7, 9, 3, 0],
  [4, 9, 1, 3, 6, 7, 2, 7, 11, 0, 8],
  [4, 9, 3, 7, 9, 6, 1, 0, 7, 7, 11, 11, 2, 8],
  [3, 7, 4, 7, 2, 8, 0, 9, 1, 11, 6]

In [46]:
dictionary = DGL.ModelClassifyNodes(model, test_data.dataset)
# actual = dictionary['alllabels']
actual = []
for l in dictionary['alllabels']:
    actual.extend(l)
    
# print(actual)
# predicted = dictionary['allpredictions']
predicted = []
for l in dictionary['allpredictions']:
    predicted.extend(l)
# print(predicted)
result = DGL.Accuracy(actual=actual, predicted=predicted)
# print(result)
print("Accuracy:", result['accuracy'])

Accuracy: 0.748103


In [47]:
cf = DGL.ConfusionMatrix(actual = actual, predicted = predicted)
print(cf)
fig = Plotly.FigureByConfusionMatrix(cf)
Plotly.Show(fig)

[[ 57   0   0   0   0   0   0   1   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0]
 [  0  50  40   0   0   0   0   0   0   0   0   0]
 [  3   0  45 100   0   0   3   0   0   5   0   0]
 [  0   0   0   0  61   0   0   1   0   0   0   0]
 [  0   0   1   0   0  10   0   0   0   0   0   0]
 [  0   0  34   0   0   0 178   0   0  17   0  22]
 [  0   0   0   0  36   0   0 210  11   0   0   0]
 [  0   0   0   0   0   0   0   0  81   0   0   0]
 [  0   0   0   0   0   0   0   0   0  49   0   0]
 [  0  20   0   0   0   0   0   0   0  11 110   0]
 [ 10   0   0   0   3  20  18  13   8   0   0  90]]
